# Postprocess predictions and create a submission

In [9]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle, INPUT_FOLDER, WORK_FOLDER
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/output', '/kaggle/input', '/kaggle/working']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [10]:
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

INPUT_FOLDER_ORIGINAL_PNG = INPUT_FOLDER / "vinbigdata-chest-xray-original-png"
INPUT_FOLDER_YOLO_OUT = WORK_FOLDER / "vbdyolo-out"
INPUT_FOLDER_BINARY = WORK_FOLDER / "vbdbinary"

## Get data from Kaggle

In [11]:
# Version notes: YOLOv5s, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15
!kaggle datasets download "witalia/vbdyolo-out-newest/version/4" -p {INPUT_FOLDER_YOLO_OUT} --unzip --force

 35% 9.00M/25.8M [00:00<00:00, 32.2MB/s]
100% 25.8M/25.8M [00:00<00:00, 85.8MB/s]


In [12]:
# Version notes: initial release (EffNet)
!kaggle datasets download "witalia/vbdbinary/version/1" -p {INPUT_FOLDER_BINARY} --unzip --force

 90% 81.0M/90.0M [00:00<00:00, 168MB/s]
100% 90.0M/90.0M [00:00<00:00, 183MB/s]


## Process YOLO output

In [13]:
def read_prediction_labels(filename: Path, image_w: int, image_h: int):
    if not filename.exists():
        return "14 1 0 0 1 1"

    labels: pd.DataFrame = pd.read_csv(filename, delimiter=" ", header=None)
    labels.columns = ["class_id", "x_centre", "y_centre", "bw", "bh", "conf"]

    # Convert YOLO format (x_centre, y_centre, bw, bh) to competition format (x_min, y_min, x_max, y_max)
    labels["x_min"] = labels["x_centre"] - labels["bw"] / 2
    labels["y_min"] = labels["y_centre"] - labels["bh"] / 2
    labels["x_max"] = labels["x_centre"] + labels["bw"] / 2
    labels["y_max"] = labels["y_centre"] + labels["bh"] / 2
    labels = labels.drop(columns=["x_centre", "y_centre", "bw", "bh"])
    # After dropping, conf column should become the second one.
    assert(labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    # Scale coordinates to image's size. Clip to make sure it's not out of bounds of the image.
    labels[["x_min", "x_max"]] = (labels[["x_min", "x_max"]] * image_w).round().astype(np.int32).clip(0, image_w - 1)
    labels[["y_min", "y_max"]] = (labels[["y_min", "y_max"]] * image_h).round().astype(np.int32).clip(0, image_h - 1)

    # Convert all rows to one prediction string
    return " ".join(labels.to_string(header=False, index=False).split())

In [14]:
results_yolo_df = pd.DataFrame(columns=["image_id", "PredictionString"])

test_metadata = pd.read_csv(INPUT_FOLDER_ORIGINAL_PNG / "test_meta.csv")
test_metadata = test_metadata.set_index("image_id").to_dict("index")

for image_id, image_dims in tqdm(test_metadata.items(), total=len(test_metadata)):
    prediction_str = read_prediction_labels(
        INPUT_FOLDER_YOLO_OUT / "labels_pred" / f"{image_id}.txt", image_dims["dim0"], image_dims["dim1"]
    )
    results_yolo_df = results_yolo_df.append({"image_id": image_id, "PredictionString": prediction_str}, ignore_index=True)

results_yolo_df.sample(10)

,image_id,PredictionString
68,069198879ff8140bab6c47c58242a0a4,14 1 0 0 1 1
2326,c7a6447eb7bd68b642972660f26c0362,13 0.200439 1008 411 1407 1002 0 0.227295 1614...
1476,80bc7a1af89f2b8b60ed33c83ae8ed5d,3 0.154785 1206 1352 2312 1888
1712,945e911b21f8a7a61e1828b1b96e9b57,3 0.156006 875 1220 2037 1733
1788,9b9d4926513c155d102818c7e746dc67,14 1 0 0 1 1
1227,6e07fab2014be723250f7897ab6e3df2,3 0.439941 1302 1461 2367 1854 0 0.453125 1644...
797,48fcc25810228823538d0392877fcce0,14 1 0 0 1 1
1729,967a55293ed97ec1447c0da5162a430e,3 0.285156 868 1424 2032 1897
1731,968735441bd2c77d7cfcf4d504b1fd5b,0 0.337891 1210 657 1551 1015
2484,d44bdbad1399ee98febd7336ef8f1de4,14 1 0 0 1 1


## Merge with Binary classifier output

In [15]:
results_binary_df = pd.read_csv(INPUT_FOLDER_BINARY / "prediction.csv")
display(results_binary_df.head())

results_df = results_yolo_df.merge(results_binary_df, on="image_id")
results_df.loc[results_df["class_name"] == "normal", "PredictionString"] = "14 1 0 0 1 1"
results_df = results_df.drop(columns=["class_name"])

results_df.to_csv(WORK_FOLDER / "submission.csv", index=False)
display(results_df.head())

,image_id,class_name
0,8bfad13f75648d94cdc8fc5f988f701f,normal
1,235d5a43e07233fa0117ed8a185c06bf,abnormal
2,bc247fbc4a986f2974c1d8b158de77bd,abnormal
3,93cdc1245a32efee2b5ec92abe175e8b,normal
4,f734d8823f31d805567d6480513003d2,normal


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,3 0.386230 1119 1429 2048 1773 0 0.503418 1499...
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,7 0.152832 691 442 854 730 0 0.158569 1110 709...


## Submit to Kaggle

In [16]:
submission_message = "YOLOv5s, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15 + binary classifier" #@param {type:"string"}
!kaggle competitions submit \
    vinbigdata-chest-xray-abnormalities-detection \
    -f {WORK_FOLDER}/submission.csv \
    -m "{submission_message}"
!sleep 10
!kaggle competitions submissions vinbigdata-chest-xray-abnormalities-detection

100% 161k/161k [00:01<00:00, 83.5kB/s]
Successfully submitted to VinBigData Chest X-ray Abnormalities DetectionWarning: Looks like you're using an outdated API Version, please consider updating (server 1.5.10 / client 1.5.4)
fileName               date                 description                                                                                          status    publicScore  privateScore  
---------------------  -------------------  ---------------------------------------------------------------------------------------------------  --------  -----------  ------------  
submission.csv         2021-03-16 22:39:27  YOLOv5s, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15 + binary classifier  complete  0.092        None          
submission.csv         2021-03-16 22:21:54  YOLOv5s, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.5, conf 0.15 + binary classifier   complete  0.092        None          
submission.csv         2021-03-16 16:33:56 